In [ ]:
from bs4 import BeautifulSoup
import urllib, os, urllib.request
import re
import requests
from mutagen.mp3 import MP3
from mutagen.id3 import ID3, TIT2
import os
import sys
import csv


lyricsPath = os.getcwd()+'\\songs_lyrics'
if not os.path.exists(lyricsPath):
    os.makedirs(lyricsPath)
lyricsPath

In [ ]:
def urlify(s):

    # Remove all non-word characters (everything except numbers and letters)
    s = re.sub(r"[^\w\s]", '', s)

    # Replace all runs of whitespace with a single dash
    s = re.sub(r"\s+", '-', s)

    return s


In [ ]:
def generateUrls():
    print("----------- generating urls -----------")
    path = 'D:\MUSIC\sounds\RAP\Eminem'
    files = []
    urls = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path):
        for file in f:
            if '.mp3' in file :
                files.append(os.path.join(r, file))
                
    for f in files:
        try :
            tags = ID3(os.path.join(r,f))
            if 'TIT2' in tags:
                #songName = urlify(tags['TIT2'].text[0])
                songName = tags['TIT2'].text[0]
                songName = re.sub(r"feat+.([^\w]|[\w])*","",songName,flags=re.IGNORECASE)
                songName = urlify(songName)
                songName = songName.replace("'",'')
                if songName[-1]=='-':
                    songName = songName[:-1]
                url = "https://genius.com/eminem-"+songName+"-lyrics"
                urls.append(url)
        except :
            tags = ID3()
    urls = list(dict.fromkeys(urls))
    urls
    urlsFile = open(os.path.expanduser(r"urls.csv"),"wb")
    for url in urls:
        urlsFile.write(bytes(url+"\n",encoding="ascii", errors='ignore'))
    urlsFile.close()

In [ ]:
def generateWorkingAndNotWorkingUrlsFile():
    print("----------- generating files -----------")
    urls = []
    notWorkingUrls = []
    workingURLS =[]
    with open(os.path.expanduser(r"urls.csv"),"r") as urlsFile:
       # urls = csv.reader(urlsFile, delimiter=' ', quotechar='|')
        for url in urlsFile:
            urls.append(url)
    urlsFile.close()

    hdr = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36',
             'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
             'Referer': 'https://cssspritegenerator.com',
             'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
             'Accept-Encoding': 'none',
             'Accept-Language': 'en-US,en;q=0.8',
             'Connection': 'keep-alive'}
    for url in urls:
        try:
            req=urllib.request.Request(url,headers=hdr)
            thePage = urllib.request.urlopen(req)
            soup = BeautifulSoup(thePage, "html.parser")
            title = soup.find('div',class_='header_with_cover_art-primary_info').h1.text
            title = re.sub(r'/',"-",title)
            workingURLS.append(url)
        except urllib.error.HTTPError as err:
            if err.code == 404:
                notWorkingUrls.append(url)
            else:
                raise
    workingURLS = list(dict.fromkeys(workingURLS))
    notWorkingUrls = list(dict.fromkeys(notWorkingUrls))
    urls = list(dict.fromkeys(urls))
    notUrlsFile = open(os.path.expanduser(r"not-working-urls.csv"),"wb")
    for url in notWorkingUrls :
        urls.remove(url)
        notUrlsFile.write(bytes(url,encoding="ascii", errors='ignore'))
    notUrlsFile.close()
    urlsFile = open(os.path.expanduser(r"working-urls.csv"),"wb")
    for url in workingURLS:
        urlsFile.write(bytes(url,encoding="ascii", errors='ignore'))
    urlsFile.close()

In [ ]:
def getWorkingUrls():
    urls = []
    i=0
    with open(os.path.expanduser(r"working-urls.csv"),"r") as urlsFile:
        for url in urlsFile:
            urls.append(url)
    urlsFile.close()
    urls.sort()
    urls = list(dict.fromkeys(urls))
    return urls

In [ ]:
def getExistingUrls():
    existingUrls = []
    if os.path.isfile(lyricsPath+r'\existing-urls.csv'):
        with open(os.path.expanduser(lyricsPath+r'\existing-urls.csv'),'r') as existingUrlsFile:
            for url in existingUrlsFile:
                url = re.sub(r'.*url : ', '',url)
                existingUrls.append(url)
        existingUrlsFile.close()
        existingUrls.sort()
        return existingUrls

In [ ]:
def scrappingWorkingUrls():
    print("----------- scrapping urls -----------")
    pathExistingUrlsFile= lyricsPath+r'\existing-urls.csv'
    pathExistingUrlsFile
    if os.path.isfile(pathExistingUrlsFile):
        existingUrlsFile = open(os.path.expanduser(pathExistingUrlsFile),'ab')
    else:
        existingUrlsFile = open(os.path.expanduser(pathExistingUrlsFile),'wb')

    hdr = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36',
             'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
             'Referer': 'https://cssspritegenerator.com',
             'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
             'Accept-Encoding': 'none',
             'Accept-Language': 'en-US,en;q=0.8',
             'Connection': 'keep-alive'}
    urls = getWorkingUrls()
    existingUrls = getExistingUrls()
    for url in urls :
        if  not url in existingUrls:
            try:
                req=urllib.request.Request(url,headers=hdr)
                thePage = urllib.request.urlopen(req)
                soup = BeautifulSoup(thePage, "lxml")
                lyrics = soup.find('div', class_="lyrics").p.text
                title = soup.find('div',class_='header_with_cover_art-primary_info').h1.text
                title = re.sub(r'/',"-",title)
                title = re.sub(r'[^\w\s]|[\-\(\)\']','',title)
                artist = soup.find('div',class_='header_with_cover_art-primary_info').h2.text.lstrip().rstrip()
                if not os.path.exists(lyricsPath+"\\"+title+".txt"):
                    songFile = open(os.path.expanduser(lyricsPath+"\\"+title+".txt"),"wb")
                    songFile.write(bytes("Title : "+title+"\nArtist : "+artist+"\nLyrics :\n"+lyrics,encoding="ascii", errors='ignore'))
                    songFile.close()
                    existingUrlsFile.write(bytes("Title : "+title+" , url : "+url,encoding="ascii", errors='ignore'))
                    print(urls[urls.index(url)])
            except urllib.error.HTTPError as err:
                raise
    existingUrlsFile.close()
    print('Done')

In [ ]:
if not os.path.isfile(r'urls.csv'):
    generateUrls()
generateWorkingAndNotWorkingUrlsFile()
scrappingWorkingUrls()